# Feature Matching

This notebook evaluates feature matching performance for a number of test scenarios.

TODO: There are images that clearly have bad homographies, but are added together nevertheless.

TODO: Refactor the code in here to use the code that the mosaics use.


# Setup


## Imports


In [ ]:
# import copy
import glob
import itertools
import os
import time
import warnings

In [ ]:
import cv2
import numpy as np
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.utils import check_random_state
import tqdm.notebook
import yaml

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# import matplotlib.patches as patches
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, raster, features, preprocess

## Settings


In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    # Filetree settings
    'test_images_dir': '../test_data/feature_matching/',
    'src_format': 'src_{:03d}.tiff',
    'dst_format': 'dst_{:03d}.tiff',
    
    # What image transformers to test
    'img_transformer_names': [
        'PassImageTransformer',
        'CleanImageTransformer',
        'LogscaleImageTransformer',
        'CLEAN_LOGSCALE_IMAGE_PIPELINE',
    ],

    # Feature matching options
    'feature_detectors': [
        ('ORB', {}),
        ('SIFT', {}),
        # Still marked as patented in the opencv version I'm using.
        # (cv2.xfeatures2d.SURF_create, {}),
        ('AKAZE', {}),
        # Brisk "works" for almost all cases, but many of them are wrong
        # ('BRISK', {}),
        # Does not seem to be fully implemented in OpenCV
        # ('FastFeatureDetector', {}),
        # Does not seem to be fully implemented in OpenCV
        # ('MSER', {}),
        # ('SimpleBlobDetector', {}),
    ],
    'feature_matchers': [
        # TODO: Explore other feature matchers.
        ('BFMatcher', {}),
        # ('FlannBasedMatcher', {}),
        # ('BFMatcher', {'k': [10,]}),
        # TODO: Try Grid-based Motion Statistics. Very fast, but more complicated.
    ],
    'image_transformers': [
        ('PassImageTransformer', {}),
        ('CLEAN_LOGSCALE_IMAGE_PIPELINE', {}),
    ],
    'other_opts_param_grid': {
        'homography_method': [
            cv2.RANSAC,
            # cv2.RHO,
            # These don't really show promise
            # cv2.LMEDS,
            # 0,
        ],
        # 'ransacReprojThreshold': np.arange(1, 10),
        # 'maxIters': [100, 1000, 2000, 10000],
        'n_matches_used': [10, 100, 500, None, ],
        'required_brightness': [0.01, ],
        'required_bright_pixel_area': [50000, ],
        # 'det_min': [0.4, ],
    },

    # Analysis parameters
    'det_min': 0.6,
    'det_max': 2.0,
    'n_images': 10000,
    'show_images': True,
}
settings.update(local_settings)

## Parse Settings


In [ ]:
# Initialize the feature detectors
feature_detectors = []
for subsettings in settings['feature_detectors']:

    if len(subsettings[1]) == 0:
        feature_detectors.append(subsettings)
        continue
    
    # Generate all permutations of values
    param_grid = subsettings[1]
    keys, values = zip(*param_grid.items())
    permutations = itertools.product(*values)
    
    list_addition = [
        (
            subsettings[0],
            dict(zip(keys, permutation))
        )
        for permutation in permutations
    ]
    feature_detectors += list_addition

In [ ]:
# Initialize the feature matchers
feature_matchers = []
for subsettings in settings['feature_matchers']:

    if len(subsettings[1]) == 0:
        feature_matchers.append(subsettings)
        continue
    
    # Generate all permutations of values
    param_grid = subsettings[1]
    keys, values = zip(*param_grid.items())
    permutations = itertools.product(*values)
    
    list_addition = [
        (
            subsettings[0],
            dict(zip(keys, permutation))
        )
        for permutation in permutations
    ]
    feature_matchers += list_addition

In [ ]:
# Initialize the image transformers
image_transformers = []
for subsettings in settings['image_transformers']:

    if len(subsettings[1]) == 0:
        image_transformers.append(subsettings)
        continue
    
    # Generate all permutations of values
    param_grid = subsettings[1]
    keys, values = zip(*param_grid.items())
    permutations = itertools.product(*values)
    
    list_addition = [
        (
            subsettings[0],
            dict(zip(keys, permutation))
        )
        for permutation in permutations
    ]
    image_transformers += list_addition

In [ ]:
# Get other options

# Generate all permutations of values
param_grid = settings['other_opts_param_grid']
keys, values = zip(*param_grid.items())
permutations = itertools.product(*values)

other_joiner_options = [
    dict(zip(keys, permutation))
    for permutation in permutations
]

# Code


In [ ]:
class JoinSearcher:

    def __init__(self, src_fp, dst_fp):

        self.src_fp = src_fp
        self.src_image = raster.Image.open(src_fp)
        self.src_img = self.src_image.img_int
        self.dst_fp = dst_fp
        self.dst_image = raster.Image.open(dst_fp)
        self.dst_img = self.dst_image.img_int

    def show_before(self, img='semitransparent_img', img_transformer=None):

        subplot_mosaic = [['dst_img', 'src_img']]
        fig = plt.figure(figsize=(20,10))
        ax_dict = fig.subplot_mosaic(subplot_mosaic)
        
        ax = ax_dict['dst_img']
        self.dst_image.show(ax=ax, img=img, img_transformer=img_transformer)
        ax.set_title('dst')
        
        ax = ax_dict['src_img']
        self.src_image.show(ax=ax, img=img, img_transformer=img_transformer)
        ax.set_title('src')
        
        plt.tight_layout()

    def show_after(self, img='semitransparent_img', img_transformer=None):
        
        # View results
        subplot_mosaic = [['warped_img', 'blended_img']]
        fig = plt.figure(figsize=(20,10))
        ax_dict = fig.subplot_mosaic(subplot_mosaic)
        
        ax = ax_dict['warped_img']
        self.warped_image.show(ax=ax, img=img, img_transformer=img_transformer)
        ax.set_title('warped')
        
        ax = ax_dict['blended_img']
        self.blended_image.show(ax=ax, img=img, img_transformer=img_transformer)
        ax.set_title('blended')
        
        plt.tight_layout()

    def warp_and_blend(self, M):
       
        # Warp image
        self.warped_img = features.ImageJoiner.warp(self.src_img, self.dst_img, M)
        self.warped_image = raster.Image(self.warped_img)

        # Blend images
        self.blended_img = features.ImageJoiner.blend(self.warped_img, self.dst_img)
        self.blended_image = raster.Image(self.blended_img)

    def grid_search(self, feature_detectors, feature_matchers, other_joiner_options, log_keys=['abs_det_M', 'bright_area']):

        n_fd = len(feature_detectors)
        n_fm = len(feature_matchers)
        n_t = len(image_transformers)
        n_oo = len(other_joiner_options)
        n_tot = n_fd * n_fm * n_t * n_oo

        rows = []
        # TODO: Somehow the number of iterations doesn't match with pbar
        with tqdm.notebook.tqdm(total=n_tot) as pbar:
            for i, fd_settings in enumerate(feature_detectors):
                for j, fm_settings in enumerate(feature_matchers):
                    for k, t_settings in enumerate(image_transformers):
                        for m, other_opts in enumerate(other_joiner_options):
    
                            image_joiner = features.ImageJoiner(
                                feature_detector=fd_settings[0],
                                feature_detector_options=fd_settings[1],
                                feature_matcher=fm_settings[0],
                                feature_matcher_options=fm_settings[1],
                                image_transformer=t_settings[0],
                                image_transformer_options=t_settings[1],
                                log_keys=log_keys,
                                **other_opts
                            )
        
                            return_code, results_ijkm, log = image_joiner.join(
                                self.src_img, self.dst_img)
        
                            # Store results
                            row = {
                                'return_code': return_code,
                                'i_fd': i,
                                'j_fm': j,
                                'k_t': k,
                                'm_oo': m,
                            }
                            # Make blanks for things we want to log no matter what
                            row_defaults = {key: np.nan for key in log_keys}
                            row.update(row_defaults)
                            # Actual update
                            row.update(results_ijkm)
                            row.update(log)
                            rows.append(row)

                            pbar.update(1)
                        pbar.update(1)
                    pbar.update(1)
                pbar.update(1)

        df = pd.DataFrame(rows)

        # Measure of how warped the image is
        df['warp_factor'] = np.abs(np.log10(np.abs(df['abs_det_M'])))

        # Identify the best set of parameters
        df['valid_M'] = df['return_code'] == 'success'
        valid_df = df.loc[df['valid_M']]
        if len(valid_df) > 0:
            best_ind = valid_df.index[valid_df['duration'].argmin()]
        else:
            best_ind = df.index[df['warp_factor'].argmin()]
        best_row = df.loc[best_ind]
        t_best_ind = pd.Timedelta(
            settings['n_images'] * best_row['duration'], unit='second')

        # Results
        gs_info = {
            'n_valid': len(valid_df),
            'best_ind': best_ind,
            't_best_ind': t_best_ind,
            'best_fd': feature_detectors[best_row['i_fd']],
            'best_fm': feature_matchers[best_row['j_fm']],
            'best_t': image_transformers[best_row['k_t']],
            'best_oo': other_joiner_options[best_row['m_oo']],
        }

        if len(valid_df) > 0:
            print(
                f'''Grid search complete.
                    {gs_info['n_valid']} valid results.
                    Best valid time was {best_row['duration']:.2g} seconds, for an estimated total time of {gs_info['t_best_ind']}.
                    The best feature detector was {gs_info['best_fd']}
                    The best feature matcher was {gs_info['best_fm']}
                    The best transformer was {gs_info['best_t']}
                    The best other options were {gs_info['best_oo']}
                '''
            )
        else:
            print(
                f'''No successes found. Closest det_min was {best_row['abs_det_M']:.3g}
                '''
            )

        self.df = df
        self.best_row = best_row
        self.gs_info = gs_info

        return df, best_row, gs_info


# Test Image Joins


## A Particular Set


In [ ]:
i = 7
src_fp = os.path.join(settings['test_images_dir'], settings['src_format'].format(i))
dst_fp = os.path.join(settings['test_images_dir'], settings['dst_format'].format(i))
js = JoinSearcher(src_fp, dst_fp)

In [ ]:
if settings['show_images']:
    js.show_before(
        img='img_int',
        img_transformer=preprocess.CLEAN_LOGSCALE_IMAGE_PIPELINE
    )

In [ ]:
df, best_row, grid_search_results = js.grid_search(feature_detectors, feature_matchers, other_joiner_options)

In [ ]:
row = best_row
row = df.loc[8]
if 'M' in row:
    js.warp_and_blend(row['M'])

    t_settings = image_transformers[row['k_t']]
    img_transformer = getattr(preprocess, t_settings[0])(**t_settings[1])

    if settings['show_images']:
        js.show_after(
            img='img_int',
            img_transformer=img_transformer,
        )

## All Sets


In [ ]:
src_fps = sorted(glob.glob(os.path.join(settings['test_images_dir'], 'src_*.tiff')))
dst_fps = sorted(glob.glob(os.path.join(settings['test_images_dir'], 'dst_*.tiff')))

In [ ]:
results = []
for i, src_fp in enumerate(src_fps):
    print(f'i = {i} / {len(src_fps)}')
    
    dst_fp = dst_fps[i]

    fc = JoinSearcher(src_fp, dst_fp)
    
    df, best_row, grid_search_results = fc.grid_search(feature_detectors, feature_matchers, other_joiner_options)
    
    df['set'] = i
    results.append(df)

## Summarize


In [ ]:
df = pd.concat(results, ignore_index=True)

In [ ]:
df['ijkm'] = 'i' + df['i_fd'].astype(str) + '_j' + df['j_fm'].astype(str) + '_k' + df['k_t'].astype(str)

In [ ]:
valid_df = df.loc[df['valid_M']]
valid_or_dark_df = df.loc[df['return_code'].isin(['success', 'dark_frame'])]

In [ ]:
n_sets = pd.unique(df['set']).size

### Overview


In [ ]:
sns.scatterplot(
    data=df,
    x='duration',
    y='warp_factor',
    hue='valid_M',
)

ax = plt.gca()
ax.axhline(np.abs(np.log10(settings['det_min'])))
ax.set_ylim(0, ax.get_ylim()[1])

### Identify Promising Parameters


In [ ]:
# Feature detectors that work across all image pairs
n_valid_sets = valid_or_dark_df.groupby('i_fd')['set'].nunique()
promising_i_fd = n_valid_sets.index[n_valid_sets==n_sets]
promising_fd = [feature_detectors[_] for _ in promising_i_fd]
promising_fd

In [ ]:
# Feature matchers that work across all image pairs
n_valid_sets = valid_or_dark_df.groupby('j_fm')['set'].nunique()
promising_fm = [feature_matchers[_] for _ in n_valid_sets.index[n_valid_sets==n_sets]]
promising_fm

In [ ]:
# Feature matchers that work across all image pairs
n_valid_sets = valid_or_dark_df.groupby('k_t')['set'].nunique()
promising_t = [image_transformers[_] for _ in n_valid_sets.index[n_valid_sets==n_sets]]
promising_t

In [ ]:
# Transform parameters that work across all image pairs
n_valid_sets = valid_or_dark_df.groupby('k_t')['set'].nunique()
promising_t_kwargs = [other_joiner_options[_] for _ in n_valid_sets.index[n_valid_sets==n_sets]]
method_map = {
    getattr(cv2, method): method
    for method in ['RANSAC', 'LMEDS', 'RHO']
}
promising_t_kwargs = [
    {
        key:(method_map[value] if key == 'method' else value)
        for key, value in t_kwargs.items()
    } for t_kwargs in promising_t_kwargs
]
promising_t_kwargs

In [ ]:
# And now the combinations that are fully good
ijkm_groups = valid_or_dark_df.groupby('ijkm')
n_valid_sets = ijkm_groups['set'].nunique()
is_good = n_valid_sets == n_sets
good_ijkms = n_valid_sets.index[is_good]

In [ ]:
if is_good.sum() > 0:
    # Convert into a dataframe
    good_df = ijkm_groups.first().loc[good_ijkms]
    good_df['duration'] = ijkm_groups['duration'].mean().loc[good_ijkms]
    good_df = good_df.sort_values('duration')
    best_row = good_df.iloc[0]

    # Print the best (quickest while still valid) combination
    print(
        feature_detectors[best_row['i_fd']],
        feature_matchers[best_row['j_fm']],
        image_transformers[best_row['k_t']],
        other_joiner_options[best_row['m_oo']]
    )
else:
    print('No single set of parameters works for all images.')

# Test Image Transforms


In [ ]:
# DEBUG
import importlib
importlib.reload(features)

In [ ]:
i = 3
fp = os.path.join(settings['test_images_dir'], settings['dst_format'].format(i))
image = raster.Image.open(fp)

In [ ]:
# We want our transforms to be smooth.
class treat_warnings_as_errors:
    def __enter__(self):
        warnings.simplefilter("error")

    def __exit__(self, exc_type, exc_val, exc_tb):
        warnings.resetwarnings()

In [ ]:
with treat_warnings_as_errors():

    transformed_imgs = []
    for img_transformer_name in settings['img_transformer_names']:

        img_transformer = getattr(preprocess, img_transformer_name)
        if callable(img_transformer):
            img_transformer = img_transformer()

        try:
            img_t = img_transformer.fit_transform(X=[image.img_int])[0]
        except NotImplementedError:
            continue
        image_t = raster.Image(img_t)

        transformed_imgs.append(image_t)

In [ ]:
if settings['show_images']:
    figsize = np.array(image.img_shape) / 240.
    ncol = len(settings['img_transformer_names'])
    subplot_mosaic = [[_, ] for _ in settings['img_transformer_names']]
    figsize[1] *= ncol
    fig = plt.figure(figsize=figsize)
    ax_dict = fig.subplot_mosaic(subplot_mosaic)

    for i, image_t in enumerate(transformed_imgs):

        img_transformer_name = settings['img_transformer_names'][i]

        ax = ax_dict[img_transformer_name]
        image_t.show(ax=ax)

        ax.set_title(img_transformer_name)

    plt.tight_layout()

# Test Feature Detection Details


In [ ]:
image_joiner = features.ImageJoiner(
    feature_detector='AKAZE',
    feature_detector_options={},
    feature_matcher='BFMatcher',
    feature_matcher_options={},
    image_transformer='CLEAN_LOGSCALE_IMAGE_PIPELINE',
    image_transformer_options={},
)

In [ ]:
dst_kp, dst_des = image_joiner.detect_and_compute(js.dst_img)
dst_pts = cv2.KeyPoint_convert(dst_kp)

In [ ]:
js.dst_image.show()
fig = plt.gcf()
ax = plt.gca()

ax.scatter(
    dst_pts[:,0],
    dst_pts[:,1],
    color='none',
    edgecolor='w',
    linewidth=3,
    s=150,
)

In [ ]:
k = 3
dst_pt = dst_pts[k]
dst_de = dst_des[k]

In [ ]:
hw = 128
px_min = max(int(dst_pt[1] - hw), 0)
px_max = min(int(dst_pt[1] + hw), js.dst_img.shape[0])
py_min = max(int(dst_pt[0] - hw), 0)
py_max = min(int(dst_pt[0] + hw), js.dst_img.shape[1])
zoom = js.dst_img[px_min:px_max, py_min:py_max]
zoom_image = raster.Image(zoom)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.gca()

zoom_image.show(
    ax=ax,
    img='img_int',
    img_transformer=image_joiner.image_transformer
)